###Remember, while the name "tool calling" implies that the model is directly performing some action, this is actually not the case! The model only generates the arguments to a tool, and actually running the tool (or not) is up to the user.

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = "sk-"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [33]:
llm.invoke("What is the weather in hyderabad")

AIMessage(content="I don't have real-time data access to provide current weather information. To check the current weather in Hyderabad, please refer to a reliable weather website or app, such as the Weather Channel, AccuWeather, or a local meteorological service.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 14, 'total_tokens': 62, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8552ec53e1', 'finish_reason': 'stop', 'logprobs': None}, id='run-aa43d9c5-e9c2-4bbe-abee-d5662f04e6d9-0', usage_metadata={'input_tokens': 14, 'output_tokens': 48, 'total_tokens': 62, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [24]:
from langchain_core.tools import tool

@tool
def sum_marks(subjects: list[int]) -> int:
    """Adds all the list of subject values passed"""
    sum = 0
    for sub in subjects:
        sum = sum+sub 
    return sum
@tool
def avg_marks(subjects: list[int]) -> int:
    """Average all the list of subject values passed"""
    num = len(subjects)
    sum = 0
    for sub in subjects:
        sum = sum+sub 
    return sum
    avg =  sum/num
    return avg

tools = [sum_marks, avg_marks]
llm_with_tools = llm.bind_tools(tools)

In [28]:
from langchain_core.messages import HumanMessage

query = "What is sum and avg marks praveen has got , his marks are 80,67,89,77,73?"

messages = [HumanMessage(query)]

ai_message =  llm_with_tools.invoke(messages)

messages.append(ai_message)

In [29]:
print(ai_message)

content='' additional_kwargs={'tool_calls': [{'id': 'call_1dSZaqgReIyqSXouKNKlJx0p', 'function': {'arguments': '{"subjects": [80, 67, 89, 77, 73]}', 'name': 'sum_marks'}, 'type': 'function'}, {'id': 'call_eBnIe3v1e6ZojXKZZ3sK8573', 'function': {'arguments': '{"subjects": [80, 67, 89, 77, 73]}', 'name': 'avg_marks'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 97, 'total_tokens': 158, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-8354484d-9c27-4e5a-ae63-3705434e3634-0' tool_calls=[{'name': 'sum_marks', 'args': {'subjects': [80, 67, 89, 77, 73]}, 'id': 'call_1dSZaqgReIyqSXouKNKlJx0p', 'type': 'tool_call'}, {'name': 'avg_marks', 'args': {'subjects': [80, 67, 89, 77, 73]}, 'id': 'call_eB

content='' additional_kwargs={'tool_calls': 

[{'id': 'call_wl368D3mPyKqp8SdTtR0f6hw', 'function': {'arguments': '{"subjects": [80, 67, 89, 77, 73]}', 'name': 'sum_marks'}, 'type': 'function'}, {'id': 'call_FufoWgy3p0sFEkQms6iTEx34', 'function': {'arguments': '{"subjects": [80, 67, 89, 77, 73]}', 'name': 'avg_marks'}, 'type': 'function'}]

, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 97, 'total_tokens': 158, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-ee28c17f-dc6c-431d-bb5d-903271865809-0' tool_calls=[{'name': 'sum_marks', 'args': {'subjects': [80, 67, 89, 77, 73]}, 'id': 'call_wl368D3mPyKqp8SdTtR0f6hw', 'type': 'tool_call'}, {'name': 'avg_marks', 'args': {'subjects': [80, 67, 89, 77, 73]}, 'id': 'call_FufoWgy3p0sFEkQms6iTEx34', 'type': 'tool_call'}] usage_metadata={'input_tokens': 97, 'output_tokens': 61, 'total_tokens': 158, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}


In [30]:
for tool_call in ai_message.tool_calls:
    # print("tool_call :::::>>>>> ",tool_call)
    selected_tool = {"sum_marks": sum_marks, "avg_marks": avg_marks}[tool_call["name"].lower()]
    # print("selected_tool ::::::>>>>> ",selected_tool)
    tool_msg = selected_tool.invoke(tool_call)
    # print("tool_msg:::::>>>>",tool_msg)
    messages.append(tool_msg)

tool_call :::::>>>>>  {'name': 'sum_marks', 'args': {'subjects': [80, 67, 89, 77, 73]}, 'id': 'call_1dSZaqgReIyqSXouKNKlJx0p', 'type': 'tool_call'}
selected_tool ::::::>>>>>  name='sum_marks' description='Adds all the list of subject values passed' args_schema=<class 'langchain_core.utils.pydantic.sum_marks'> func=<function sum_marks at 0x1149a7c40>
tool_msg:::::>>>> content='386' name='sum_marks' tool_call_id='call_1dSZaqgReIyqSXouKNKlJx0p'
tool_call :::::>>>>>  {'name': 'avg_marks', 'args': {'subjects': [80, 67, 89, 77, 73]}, 'id': 'call_eBnIe3v1e6ZojXKZZ3sK8573', 'type': 'tool_call'}
selected_tool ::::::>>>>>  name='avg_marks' description='Average all the list of subject values passed' args_schema=<class 'langchain_core.utils.pydantic.avg_marks'> func=<function avg_marks at 0x1149a42c0>
tool_msg:::::>>>> content='386' name='avg_marks' tool_call_id='call_eBnIe3v1e6ZojXKZZ3sK8573'


In [34]:
messages

[HumanMessage(content='What is sum and avg marks praveen has got , his marks are 80,67,89,77,73?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1dSZaqgReIyqSXouKNKlJx0p', 'function': {'arguments': '{"subjects": [80, 67, 89, 77, 73]}', 'name': 'sum_marks'}, 'type': 'function'}, {'id': 'call_eBnIe3v1e6ZojXKZZ3sK8573', 'function': {'arguments': '{"subjects": [80, 67, 89, 77, 73]}', 'name': 'avg_marks'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 97, 'total_tokens': 158, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8354484d-9c27-4e5a-ae63-3705434e3634-0', tool_calls=[{'name': 'sum_marks', 'args': {'subjects': [80, 

In [32]:
print(llm_with_tools.invoke(messages))

content="Praveen's total marks are 386, and his average marks are 77.2." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 174, 'total_tokens': 195, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None} id='run-897631b3-edbe-40f5-80ee-ec621b85a830-0' usage_metadata={'input_tokens': 174, 'output_tokens': 21, 'total_tokens': 195, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}


In [10]:
print(llm_with_tools.invoke(messages).content)

Praveen's total marks are 386, and his average marks are also 77.2.


In [11]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

examples = [
    HumanMessage("What's the sum of following marks 90,60,75,80", name="example_user"),
    AIMessage("",name="example_assistant",tool_calls=[
        {"name": "sum_marks", "args": {"x": [90,60,75,80] }, "id": "1"}
    ]),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "sum_marks", "args": {"x": [90,60,75,80] }, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The sum of follwoing marks is 305",
        name="example_assistant",
    ),

]

system = """You are bad at math but are an expert at using a calculator. 
Use past tool usage as an example of how to correctly use the tools."""

few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

chain = {"query":RunnablePassthrough()} | few_shot_prompt | llm_with_tools



In [12]:
chain.invoke("Whats sum marks 40,60,70")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gTa46TWfIofN9Xeiu8BUt6UL', 'function': {'arguments': '{"subjects":[40,60,70]}', 'name': 'sum_marks'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 221, 'total_tokens': 239, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'text_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a86b109c-a09e-415a-9383-6d3a0c6468ce-0', tool_calls=[{'name': 'sum_marks', 'args': {'subjects': [40, 60, 70]}, 'id': 'call_gTa46TWfIofN9Xeiu8BUt6UL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 221, 'output_tokens': 18, 'total_tokens': 239, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
chain.invoke("Whats sum marks 40,60,70").tool_calls

[{'name': 'sum_marks',
  'args': {'subjects': [40, 60, 70]},
  'id': 'call_vlKgmqy8PnFrWUUGhipDz5y0',
  'type': 'tool_call'}]

In [14]:
ai_message = chain.invoke("Whats sum marks 40,60,70").tool_calls
ai_message

[{'name': 'sum_marks',
  'args': {'subjects': [40, 60, 70]},
  'id': 'call_T3Vw84MZ3j1qKSSY402O867V',
  'type': 'tool_call'}]

In [15]:
for tool_call in ai_message:
    selected_tool = {"sum_marks": sum_marks, "avg_marks": avg_marks}[tool_call["name"].lower()]
    print("selected_tool",selected_tool)
    tool_msg = selected_tool.invoke(tool_call)
    print(tool_msg)

selected_tool name='sum_marks' description='Adds all the list of subject values passed' args_schema=<class 'langchain_core.utils.pydantic.sum_marks'> func=<function sum_marks at 0x114941a80>
content='170' name='sum_marks' tool_call_id='call_T3Vw84MZ3j1qKSSY402O867V'


#How to pass run time values to tools

In [35]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [37]:
from typing import List

from langchain_core.tools import InjectedToolArg, tool
from typing_extensions import Annotated

user_to_pets = {}

@tool(parse_docstring=True)
def update_favorite_pets(pets:List[str],user_id:Annotated[str,InjectedToolArg]) -> None:
    """Add the list of favorite pets.

    Args:
        pets: List of favorite pets to set.
        user_id: User's ID.
    """
    user_to_pets[user_id] = pets

@tool(parse_docstring=True)
def delete_favorite_pets(user_id: Annotated[str, InjectedToolArg]) -> None:
    """Delete the list of favorite pets.

    Args:
        user_id: User's ID.
    """
    if user_id in user_to_pets:
        del user_to_pets[user_id]


@tool(parse_docstring=True)
def list_favorite_pets(user_id: Annotated[str, InjectedToolArg]) -> None:
    """List favorite pets if any.

    Args:
        user_id: User's ID.
    """
    return user_to_pets.get(user_id, [])

 

In [22]:
update_favorite_pets.get_input_schema().schema()

{'description': 'Add the list of favorite pets.',
 'properties': {'pets': {'description': 'List of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'},
  'user_id': {'description': "User's ID.",
   'title': 'User Id',
   'type': 'string'}},
 'required': ['pets', 'user_id'],
 'title': 'update_favorite_pets',
 'type': 'object'}

In [23]:
update_favorite_pets.tool_call_schema.schema()

{'description': 'Add the list of favorite pets.',
 'properties': {'pets': {'description': 'List of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'}},
 'required': ['pets'],
 'title': 'update_favorite_pets',
 'type': 'object'}

In [24]:
user_id = "123"
update_favorite_pets.invoke({"pets": ["lizard", "dog"], "user_id": user_id})
print(user_to_pets)
print(list_favorite_pets.invoke({"user_id": user_id}))

{'123': ['lizard', 'dog']}
['lizard', 'dog']


In [38]:
tools = [
    update_favorite_pets,
    delete_favorite_pets,
    list_favorite_pets,
]
llm_with_tools = llm.bind_tools(tools)
ai_msg = llm_with_tools.invoke("my favorite animals are cats and parrots")
ai_msg.tool_calls

[{'name': 'update_favorite_pets',
  'args': {'pets': ['cats', 'parrots']},
  'id': 'call_IBmcGKZrIPH3B5xmqIKrcGCx',
  'type': 'tool_call'}]

In [26]:
from copy import deepcopy

from langchain_core.runnables import chain

@chain
def inject_user_id(ai_msg):
    tool_calls = []
    for tool_call in ai_msg.tool_calls:
        tool_call_copy = deepcopy(tool_call)
        print("tool_call_copy \n")
        print(tool_call_copy)
        print("tool_call_copy \n")
        tool_call_copy["args"]["user_id"] = user_id
        tool_calls.append(tool_call_copy)
    return tool_calls

inject_user_id.invoke(ai_msg)

tool_call_copy 

{'name': 'update_favorite_pets', 'args': {'pets': ['cats', 'parrots']}, 'id': 'call_FykX07b4MgIu5zfksYsdp3oV', 'type': 'tool_call'}
tool_call_copy 



[{'name': 'update_favorite_pets',
  'args': {'pets': ['cats', 'parrots'], 'user_id': '123'},
  'id': 'call_FykX07b4MgIu5zfksYsdp3oV',
  'type': 'tool_call'}]

In [39]:
tools

[StructuredTool(name='update_favorite_pets', description='Add the list of favorite pets.', args_schema=<class 'langchain_core.utils.pydantic.update_favorite_pets'>, func=<function update_favorite_pets at 0x114e20b80>),
 StructuredTool(name='delete_favorite_pets', description='Delete the list of favorite pets.', args_schema=<class 'langchain_core.utils.pydantic.delete_favorite_pets'>, func=<function delete_favorite_pets at 0x114e20ae0>),
 StructuredTool(name='list_favorite_pets', description='List favorite pets if any.', args_schema=<class 'langchain_core.utils.pydantic.list_favorite_pets'>, func=<function list_favorite_pets at 0x114e20a40>)]

In [27]:
tool_map = {tool.name: tool for tool in tools}
tool_map

{'update_favorite_pets': StructuredTool(name='update_favorite_pets', description='Add the list of favorite pets.', args_schema=<class 'langchain_core.utils.pydantic.update_favorite_pets'>, func=<function update_favorite_pets at 0x115a694e0>),
 'delete_favorite_pets': StructuredTool(name='delete_favorite_pets', description='Delete the list of favorite pets.', args_schema=<class 'langchain_core.utils.pydantic.delete_favorite_pets'>, func=<function delete_favorite_pets at 0x115a4eac0>),
 'list_favorite_pets': StructuredTool(name='list_favorite_pets', description='List favorite pets if any.', args_schema=<class 'langchain_core.utils.pydantic.list_favorite_pets'>, func=<function list_favorite_pets at 0x115a4e5c0>)}

In [28]:
@chain
def tool_router(tool_call):
    return tool_map[tool_call["name"]]

In [29]:
chain = llm_with_tools | inject_user_id | tool_router.map()
chain.invoke("my favorite animals are cats and parrots")

tool_call_copy 

{'name': 'update_favorite_pets', 'args': {'pets': ['cats', 'parrots']}, 'id': 'call_IRJnGXeWlixaR2wS8uT9Ts7c', 'type': 'tool_call'}
tool_call_copy 



[ToolMessage(content='null', name='update_favorite_pets', tool_call_id='call_IRJnGXeWlixaR2wS8uT9Ts7c')]

In [30]:
user_to_pets

{'123': ['cats', 'parrots']}

In [31]:
from langchain_core.tools import BaseTool
from pydantic import BaseModel, Field


class UpdateFavoritePetsSchema(BaseModel):
    """Update list of favorite pets"""

    pets: List[str] = Field(..., description="List of favorite pets to set.")
    user_id: Annotated[str, InjectedToolArg] = Field(..., description="User's ID.")


@tool(args_schema=UpdateFavoritePetsSchema)
def update_favorite_pets(pets, user_id):
    user_to_pets[user_id] = pets


In [32]:

update_favorite_pets.get_input_schema().schema()

{'description': 'Update list of favorite pets',
 'properties': {'pets': {'description': 'List of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'},
  'user_id': {'description': "User's ID.",
   'title': 'User Id',
   'type': 'string'}},
 'required': ['pets', 'user_id'],
 'title': 'UpdateFavoritePetsSchema',
 'type': 'object'}

In [33]:
update_favorite_pets.tool_call_schema.schema()

{'description': 'Update list of favorite pets',
 'properties': {'pets': {'description': 'List of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'}},
 'required': ['pets'],
 'title': 'update_favorite_pets',
 'type': 'object'}